# Tournament Selection

Demonstrates tournament selection for genetic algorithms. Shows how to select parents from a population based on fitness, with configurable selection pressure.

## 🔧 Configuration

**Modify these parameters to experiment with different setups:**

In [1]:
# 🔧 CONFIGURATION - Modify these parameters
POPULATION_SIZE = 100
LAYER_SIZES = [1, 100, 200, 3]  # Input -> Hidden1 -> Hidden2 -> Output
ACTIVATIONS = [1, 1, 2]         # [ReLU, ReLU, Sigmoid]
TOURNAMENT_SIZE = 3             # Tournament size (2-7 typical)
NUM_PARENTS = 50                # Number of parents to select
RANDOM_SEED = 42                # For reproducibility

# 📝 Tournament Size Effects:
# Small (2-3): Lower selection pressure, more diversity
# Medium (4-5): Balanced selection pressure
# Large (6-7): High selection pressure, less diversity

# 🎯 Selection Ratio:
# 50%: Half population selected as parents
# 25%: Quarter population (elitist)
# 75%: Three quarters (diverse)

print(f"📊 Configuration:")
print(f"   Population size: {POPULATION_SIZE}")
print(f"   Architecture: {' -> '.join(map(str, LAYER_SIZES))}")
print(f"   Tournament size: {TOURNAMENT_SIZE}")
print(f"   Parents to select: {NUM_PARENTS} ({NUM_PARENTS/POPULATION_SIZE*100:.0f}%)")
print(f"   Random seed: {RANDOM_SEED}")

📊 Configuration:
   Population size: 100
   Architecture: 1 -> 100 -> 200 -> 3
   Tournament size: 3
   Parents to select: 50 (50%)
   Random seed: 42


In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import sys
sys.path.append('../../')
import numba_ga

# Import tournament selection
from numba_ga import tournament_selection, initialize_population

## 1. Architecture Analysis

In [3]:
# Convert to numpy arrays for numba compatibility
layer_sizes = np.array(LAYER_SIZES, dtype=np.int64)
activations = np.array(ACTIVATIONS, dtype=np.int64)

# Calculate architecture properties
total_params = numba_ga.get_total_parameters(layer_sizes)
activation_names = [numba_ga.get_activation_name(a) for a in activations]

print(f"🧠 Network Architecture Analysis:")
print(f"   Layers: {len(layer_sizes)}")
print(f"   Architecture: {' -> '.join(map(str, layer_sizes))}")
print(f"   Activations: {' -> '.join(activation_names)}")
print(f"   Total parameters: {total_params:,}")
print(f"   Memory per individual: ~{total_params * 8 / 1024:.1f} KB")
print(f"   Population memory: ~{total_params * 8 * POPULATION_SIZE / (1024*1024):.1f} MB")

print(f"\n🏆 Tournament Selection Setup:")
print(f"   Tournament size: {TOURNAMENT_SIZE}")
print(f"   Selection pressure: {'Low' if TOURNAMENT_SIZE <= 2 else 'Medium' if TOURNAMENT_SIZE <= 4 else 'High'}")
print(f"   Parents selected: {NUM_PARENTS}/{POPULATION_SIZE} ({NUM_PARENTS/POPULATION_SIZE*100:.0f}%)")
print(f"   Selection ratio: {'Elitist' if NUM_PARENTS/POPULATION_SIZE < 0.3 else 'Balanced' if NUM_PARENTS/POPULATION_SIZE < 0.7 else 'Diverse'}")

🧠 Network Architecture Analysis:
   Layers: 4
   Architecture: 1 -> 100 -> 200 -> 3
   Activations: relu -> relu -> sigmoid
   Total parameters: 21,306
   Memory per individual: ~166.5 KB
   Population memory: ~16.3 MB

🏆 Tournament Selection Setup:
   Tournament size: 3
   Selection pressure: Medium
   Parents selected: 50/100 (50%)
   Selection ratio: Balanced


## 2. Population Initialization

In [4]:
print(f"🧬 Initializing population of {POPULATION_SIZE} individuals...")

# Initialize population with configured parameters
population = initialize_population(POPULATION_SIZE, layer_sizes, seed=RANDOM_SEED)

print(f"\n👥 Population Properties:")
print(f"   Shape: {population.shape}")
print(f"   Memory usage: {population.nbytes / (1024*1024):.2f} MB")
print(f"   Data type: {population.dtype}")

# Population diversity analysis
all_weights = population.flatten()
print(f"\n📊 Weight Distribution Across Population:")
print(f"   Total weights: {len(all_weights):,}")
print(f"   Mean: {np.mean(all_weights):.6f}")
print(f"   Std: {np.std(all_weights):.6f}")
print(f"   Range: {np.min(all_weights):.3f} to {np.max(all_weights):.3f}")

# Check population diversity (no identical individuals)
unique_individuals = len(np.unique(population, axis=0))
print(f"\n🎲 Population Diversity:")
print(f"   Unique individuals: {unique_individuals}/{POPULATION_SIZE}")
print(f"   Diversity: {'Perfect ✅' if unique_individuals == POPULATION_SIZE else 'Duplicates found ❌'}")

🧬 Initializing population of 100 individuals...

👥 Population Properties:
   Shape: (100, 21306)
   Memory usage: 16.26 MB
   Data type: float64

📊 Weight Distribution Across Population:
   Total weights: 2,130,600
   Mean: -0.000278
   Std: 0.300222
   Range: -1.449 to 1.448

🎲 Population Diversity:
   Unique individuals: 100/100
   Diversity: Perfect ✅


## 3. Fitness Score Generation

In [5]:
# Generate realistic fitness scores for demonstration
# We'll create a mix of patterns to show selection behavior

np.random.seed(RANDOM_SEED)

# Create hierarchical fitness with some noise
base_fitness = np.linspace(0.8, 1.3, POPULATION_SIZE)  # Linear progression
noise = np.random.normal(0, 0.05, POPULATION_SIZE)     # Add realistic noise
fitness_scores = base_fitness + noise

# Ensure all fitness scores are positive
fitness_scores = np.maximum(fitness_scores, 0.1)

print(f"💰 Fitness Score Analysis:")
print(f"   Fitness range: {np.min(fitness_scores):.3f} to {np.max(fitness_scores):.3f}")
print(f"   Mean fitness: {np.mean(fitness_scores):.3f}")
print(f"   Std deviation: {np.std(fitness_scores):.3f}")

# Convert to returns for easier interpretation
returns = (fitness_scores - 1.0) * 100
print(f"\n📈 Return Analysis:")
print(f"   Best return: {np.max(returns):.2f}%")
print(f"   Worst return: {np.min(returns):.2f}%")
print(f"   Mean return: {np.mean(returns):.2f}%")
print(f"   Profitable individuals: {np.sum(returns > 0)}/{POPULATION_SIZE} ({np.sum(returns > 0)/POPULATION_SIZE*100:.1f}%)")

💰 Fitness Score Analysis:
   Fitness range: 0.770 to 1.308
   Mean fitness: 1.045
   Std deviation: 0.155

📈 Return Analysis:
   Best return: 30.80%
   Worst return: -23.00%
   Mean return: 4.48%
   Profitable individuals: 60/100 (60.0%)


## 4. Tournament Selection Execution

In [6]:
print(f"🏆 Running tournament selection...")
print(f"   Tournament size: {TOURNAMENT_SIZE}")
print(f"   Selecting {NUM_PARENTS} parents from {POPULATION_SIZE} individuals")

# Warm up JIT compilation
print(f"\n⚡ Warming up JIT compilation...")
_ = tournament_selection(population[:5], fitness_scores[:5], TOURNAMENT_SIZE, 2, seed=RANDOM_SEED)
print(f"   JIT compilation complete ✅")

# Run tournament selection with timing
print(f"\n📊 Running tournament selection...")
start_time = time.time()

selected_parents, selected_indices = tournament_selection(
    population, fitness_scores, TOURNAMENT_SIZE, NUM_PARENTS, seed=RANDOM_SEED
)

selection_time = time.time() - start_time

print(f"\n⏱️ Performance Results:")
print(f"   Selection time: {selection_time:.6f} seconds")
print(f"   Parents per second: {NUM_PARENTS / selection_time:.0f}")
print(f"   Time per parent: {selection_time / NUM_PARENTS * 1000:.3f} ms")

print(f"\n🎯 Selection Results:")
print(f"   Selected parents shape: {selected_parents.shape}")
print(f"   Selected indices shape: {selected_indices.shape}")
print(f"   Selected indices range: {np.min(selected_indices)} to {np.max(selected_indices)}")
print(f"   Unique parents selected: {len(np.unique(selected_indices))}")

# Analyze selection bias
selected_fitness = fitness_scores[selected_indices]
print(f"\n📊 Selection Quality:")
print(f"   Population mean fitness: {np.mean(fitness_scores):.3f}")
print(f"   Selected mean fitness: {np.mean(selected_fitness):.3f}")
print(f"   Selection bias: {((np.mean(selected_fitness) / np.mean(fitness_scores)) - 1) * 100:.1f}%")
print(f"   Best individual selected: {'✅' if np.argmax(fitness_scores) in selected_indices else '❌'}")

🏆 Running tournament selection...
   Tournament size: 3
   Selecting 50 parents from 100 individuals

⚡ Warming up JIT compilation...
   JIT compilation complete ✅

📊 Running tournament selection...

⏱️ Performance Results:
   Selection time: 0.000708 seconds
   Parents per second: 70611
   Time per parent: 0.014 ms

🎯 Selection Results:
   Selected parents shape: (50, 21306)
   Selected indices shape: (50,)
   Selected indices range: 21 to 99
   Unique parents selected: 33

📊 Selection Quality:
   Population mean fitness: 1.045
   Selected mean fitness: 1.189
   Selection bias: 13.8%
   Best individual selected: ✅


## 5. Selection Analysis & Visualization

In [7]:
# Create comprehensive selection analysis visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Population Fitness Distribution',
        'Selection Frequency by Individual',
        'Selected vs Population Fitness',
        'Selection Bias Analysis'
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Population fitness histogram
fig.add_trace(
    go.Histogram(
        x=fitness_scores,
        nbinsx=20,
        name='Population Fitness',
        marker_color='lightblue',
        opacity=0.7
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=selected_fitness,
        nbinsx=20,
        name='Selected Fitness',
        marker_color='orange',
        opacity=0.7
    ),
    row=1, col=1
)

# 2. Selection frequency
selection_counts = np.bincount(selected_indices, minlength=POPULATION_SIZE)
individual_ids = np.arange(POPULATION_SIZE)

fig.add_trace(
    go.Bar(
        x=individual_ids,
        y=selection_counts,
        name='Selection Count',
        marker_color=fitness_scores,
        marker_colorscale='RdYlGn',
        marker_showscale=False
    ),
    row=1, col=2
)

# 3. Scatter plot: selected vs population
colors = ['red' if i in selected_indices else 'lightgray' for i in range(POPULATION_SIZE)]
sizes = [8 if i in selected_indices else 4 for i in range(POPULATION_SIZE)]

fig.add_trace(
    go.Scatter(
        x=individual_ids,
        y=fitness_scores,
        mode='markers',
        name='Population',
        marker=dict(color=colors, size=sizes),
        showlegend=False
    ),
    row=2, col=1
)

# Add mean lines
fig.add_hline(y=np.mean(fitness_scores), line_dash="dash", line_color="blue", 
              annotation_text="Population Mean", row=2, col=1)
fig.add_hline(y=np.mean(selected_fitness), line_dash="dash", line_color="orange", 
              annotation_text="Selected Mean", row=2, col=1)

# 4. Selection bias by fitness quartiles
quartiles = np.percentile(fitness_scores, [25, 50, 75, 100])
quartile_labels = ['Bottom 25%', '25-50%', '50-75%', 'Top 25%']
quartile_counts = []

for i in range(4):
    if i == 0:
        mask = fitness_scores <= quartiles[0]
    elif i == 3:
        mask = fitness_scores > quartiles[2]
    else:
        mask = (fitness_scores > quartiles[i-1]) & (fitness_scores <= quartiles[i])
    
    quartile_individuals = np.where(mask)[0]
    selected_from_quartile = np.sum(np.isin(selected_indices, quartile_individuals))
    quartile_counts.append(selected_from_quartile)

fig.add_trace(
    go.Bar(
        x=quartile_labels,
        y=quartile_counts,
        name='Selections by Quartile',
        marker_color=['red', 'orange', 'yellow', 'green']
    ),
    row=2, col=2
)

# Update layout
fig.update_xaxes(title_text="Fitness Score", row=1, col=1)
fig.update_xaxes(title_text="Individual ID", row=1, col=2)
fig.update_xaxes(title_text="Individual ID", row=2, col=1)
fig.update_xaxes(title_text="Fitness Quartile", row=2, col=2)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Times Selected", row=1, col=2)
fig.update_yaxes(title_text="Fitness Score", row=2, col=1)
fig.update_yaxes(title_text="Selections", row=2, col=2)

fig.update_layout(
    title=f"Tournament Selection Analysis - Size {TOURNAMENT_SIZE}, {NUM_PARENTS}/{POPULATION_SIZE} Parents",
    height=800,
    width=1000,
    showlegend=True
)

fig.show()

# Print quartile analysis
print(f"\n📊 Selection by Fitness Quartiles:")
for i, (label, count) in enumerate(zip(quartile_labels, quartile_counts)):
    percentage = count / NUM_PARENTS * 100
    print(f"   {label}: {count}/{NUM_PARENTS} ({percentage:.1f}%)")


📊 Selection by Fitness Quartiles:
   Bottom 25%: 1/50 (2.0%)
   25-50%: 5/50 (10.0%)
   50-75%: 12/50 (24.0%)
   Top 25%: 32/50 (64.0%)


## 6. Tournament Size Comparison

In [8]:
# Compare different tournament sizes
print(f"🔬 Tournament Size Comparison")
print(f"   Testing different tournament sizes to show selection pressure effects")

tournament_sizes = [1, 2, 3, 5, 7]
comparison_results = []

for k in tournament_sizes:
    # Run selection with this tournament size
    parents, indices = tournament_selection(
        population, fitness_scores, k, NUM_PARENTS, seed=RANDOM_SEED
    )
    
    selected_fitness_k = fitness_scores[indices]
    mean_selected = np.mean(selected_fitness_k)
    selection_bias = ((mean_selected / np.mean(fitness_scores)) - 1) * 100
    
    # Count selections from top half
    top_half_indices = np.where(fitness_scores >= np.median(fitness_scores))[0]
    top_half_selections = np.sum(np.isin(indices, top_half_indices))
    top_half_percentage = top_half_selections / NUM_PARENTS * 100
    
    comparison_results.append({
        'tournament_size': k,
        'mean_fitness': mean_selected,
        'bias_percentage': selection_bias,
        'top_half_percentage': top_half_percentage
    })
    
    print(f"   Tournament {k}: Bias {selection_bias:.1f}%, Top half {top_half_percentage:.1f}%")

# Visualize tournament size effects
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Selection Bias vs Tournament Size', 'Top Half Selections vs Tournament Size']
)

tournament_sizes_plot = [r['tournament_size'] for r in comparison_results]
bias_percentages = [r['bias_percentage'] for r in comparison_results]
top_half_percentages = [r['top_half_percentage'] for r in comparison_results]

fig.add_trace(
    go.Scatter(
        x=tournament_sizes_plot,
        y=bias_percentages,
        mode='markers+lines',
        name='Selection Bias',
        marker=dict(color='blue', size=8),
        line=dict(color='blue')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tournament_sizes_plot,
        y=top_half_percentages,
        mode='markers+lines',
        name='Top Half %',
        marker=dict(color='green', size=8),
        line=dict(color='green')
    ),
    row=1, col=2
)

# Add reference lines
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=50, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_xaxes(title_text="Tournament Size", row=1, col=1)
fig.update_xaxes(title_text="Tournament Size", row=1, col=2)
fig.update_yaxes(title_text="Selection Bias (%)", row=1, col=1)
fig.update_yaxes(title_text="Top Half Selections (%)", row=1, col=2)

fig.update_layout(
    title="Tournament Size Effects on Selection Pressure",
    height=400,
    width=1000,
    showlegend=False
)

fig.show()

print(f"\n📈 Selection Pressure Analysis:")
print(f"   Tournament size 1: Random selection (~50% from top half)")
print(f"   Tournament size 2: Moderate bias toward fitness")
print(f"   Tournament size 5+: Strong bias toward high fitness")
print(f"   Current setting ({TOURNAMENT_SIZE}): {top_half_percentages[tournament_sizes.index(TOURNAMENT_SIZE)]:.1f}% from top half")

🔬 Tournament Size Comparison
   Testing different tournament sizes to show selection pressure effects
   Tournament 1: Bias 0.7%, Top half 58.0%
   Tournament 2: Bias 8.7%, Top half 80.0%
   Tournament 3: Bias 13.8%, Top half 88.0%
   Tournament 5: Bias 16.8%, Top half 92.0%
   Tournament 7: Bias 18.8%, Top half 98.0%



📈 Selection Pressure Analysis:
   Tournament size 1: Random selection (~50% from top half)
   Tournament size 2: Moderate bias toward fitness
   Tournament size 5+: Strong bias toward high fitness
   Current setting (3): 88.0% from top half


## 7. Performance Scaling

In [9]:
# Test performance with different population and parent sizes
print(f"⚡ Performance Scaling Analysis")
print(f"   Testing tournament selection performance with different sizes")

performance_tests = [
    (50, 10),
    (100, 25),
    (200, 50),
    (500, 100)
]

performance_results = []

for pop_size, num_parents in performance_tests:
    # Create test population
    test_pop = initialize_population(pop_size, layer_sizes, seed=1)
    test_fitness = np.random.random(pop_size).astype(np.float64)
    
    # Warm up JIT for this size
    _ = tournament_selection(test_pop[:5], test_fitness[:5], TOURNAMENT_SIZE, 2, seed=1)
    
    # Time the selection
    start_time = time.time()
    for _ in range(10):  # Multiple runs for better timing
        _ = tournament_selection(test_pop, test_fitness, TOURNAMENT_SIZE, num_parents, seed=1)
    avg_time = (time.time() - start_time) / 10
    
    selections_per_sec = num_parents / avg_time
    performance_results.append({
        'pop_size': pop_size,
        'num_parents': num_parents,
        'time': avg_time,
        'selections_per_sec': selections_per_sec
    })
    
    print(f"   Pop {pop_size}, Parents {num_parents}: {avg_time*1000:.2f}ms ({selections_per_sec:.0f} selections/s)")

# Visualize performance scaling
pop_sizes = [r['pop_size'] for r in performance_results]
times = [r['time'] for r in performance_results]
throughputs = [r['selections_per_sec'] for r in performance_results]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Selection Time vs Population Size', 'Throughput vs Population Size']
)

fig.add_trace(
    go.Scatter(
        x=pop_sizes,
        y=times,
        mode='markers+lines',
        name='Selection Time',
        marker=dict(color='blue', size=8),
        line=dict(color='blue')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=pop_sizes,
        y=throughputs,
        mode='markers+lines',
        name='Throughput',
        marker=dict(color='green', size=8),
        line=dict(color='green')
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Population Size", row=1, col=1)
fig.update_xaxes(title_text="Population Size", row=1, col=2)
fig.update_yaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_yaxes(title_text="Selections/Second", row=1, col=2)

fig.update_layout(
    title="Tournament Selection Performance Scaling",
    height=400,
    width=1000,
    showlegend=False
)

fig.show()

print(f"\n📊 Performance Summary:")
print(f"   Tournament selection scales excellently with population size")
print(f"   Current configuration: {NUM_PARENTS/selection_time:.0f} selections/second")
print(f"   Ready for large-scale genetic algorithm operations")

⚡ Performance Scaling Analysis
   Testing tournament selection performance with different sizes
   Pop 50, Parents 10: 0.14ms (71332 selections/s)
   Pop 100, Parents 25: 0.32ms (78911 selections/s)
   Pop 200, Parents 50: 0.33ms (153267 selections/s)
   Pop 500, Parents 100: 0.65ms (154612 selections/s)



📊 Performance Summary:
   Tournament selection scales excellently with population size
   Current configuration: 70611 selections/second
   Ready for large-scale genetic algorithm operations


## 8. Summary & Next Steps

In [10]:
print(f"📊 TOURNAMENT SELECTION SUMMARY")
print(f"=" * 50)

print(f"\n🔧 Configuration Used:")
print(f"   Population size: {POPULATION_SIZE}")
print(f"   Architecture: {' -> '.join(map(str, LAYER_SIZES))}")
print(f"   Tournament size: {TOURNAMENT_SIZE}")
print(f"   Parents selected: {NUM_PARENTS} ({NUM_PARENTS/POPULATION_SIZE*100:.0f}%)")
print(f"   Total parameters: {total_params:,}")

print(f"\n🏆 Selection Results:")
selection_bias = ((np.mean(selected_fitness) / np.mean(fitness_scores)) - 1) * 100
print(f"   Selection bias: {selection_bias:.1f}% toward higher fitness")
print(f"   Population mean fitness: {np.mean(fitness_scores):.3f}")
print(f"   Selected mean fitness: {np.mean(selected_fitness):.3f}")
print(f"   Best individual selected: {'✅' if np.argmax(fitness_scores) in selected_indices else '❌'}")
print(f"   Unique parents: {len(np.unique(selected_indices))}/{NUM_PARENTS}")

print(f"\n⚡ Performance Metrics:")
print(f"   Selection time: {selection_time:.6f} seconds")
print(f"   Throughput: {NUM_PARENTS / selection_time:.0f} selections/second")
print(f"   Time per selection: {selection_time / NUM_PARENTS * 1000:.3f} ms")

print(f"\n📊 Selection Pressure Analysis:")
top_half_selected = np.sum(selected_indices >= POPULATION_SIZE // 2)
print(f"   Tournament size {TOURNAMENT_SIZE}: {'Low' if TOURNAMENT_SIZE <= 2 else 'Medium' if TOURNAMENT_SIZE <= 4 else 'High'} selection pressure")
print(f"   Top half selections: {top_half_selected}/{NUM_PARENTS} ({top_half_selected/NUM_PARENTS*100:.1f}%)")
print(f"   Expected for random: ~50%")

print(f"\n🧬 Genetic Algorithm Readiness:")
print(f"   ✅ Tournament selection working perfectly")
print(f"   ✅ Configurable selection pressure")
print(f"   ✅ High-performance implementation")
print(f"   ✅ Bias toward fitness confirmed")
print(f"   ✅ Scalable to large populations")

print(f"\n📈 Next Steps:")
print(f"   1. Crossover operations (breed selected parents)")
print(f"   2. Mutation operations (add genetic diversity)")
print(f"   3. Population replacement strategies")
print(f"   4. Complete evolutionary loop")
print(f"   5. Multi-generational optimization")

print(f"\n✅ TOURNAMENT SELECTION COMPLETE!")
print(f"   🎯 Parents selected based on fitness")
print(f"   📊 Selection pressure configurable and working")
print(f"   ⚡ Performance optimized for genetic algorithms")
print(f"   🔬 Ready for crossover and mutation operations")

📊 TOURNAMENT SELECTION SUMMARY

🔧 Configuration Used:
   Population size: 100
   Architecture: 1 -> 100 -> 200 -> 3
   Tournament size: 3
   Parents selected: 50 (50%)
   Total parameters: 21,306

🏆 Selection Results:
   Selection bias: 13.8% toward higher fitness
   Population mean fitness: 1.045
   Selected mean fitness: 1.189
   Best individual selected: ✅
   Unique parents: 33/50

⚡ Performance Metrics:
   Selection time: 0.000708 seconds
   Throughput: 70611 selections/second
   Time per selection: 0.014 ms

📊 Selection Pressure Analysis:
   Tournament size 3: Medium selection pressure
   Top half selections: 43/50 (86.0%)
   Expected for random: ~50%

🧬 Genetic Algorithm Readiness:
   ✅ Tournament selection working perfectly
   ✅ Configurable selection pressure
   ✅ High-performance implementation
   ✅ Bias toward fitness confirmed
   ✅ Scalable to large populations

📈 Next Steps:
   1. Crossover operations (breed selected parents)
   2. Mutation operations (add genetic diversity